<a href="https://colab.research.google.com/github/MukulKirtiVerma/Negtive-Positive-Word-List/blob/master/Amazon_Sentiment_Analysis_with_word_Polarity_Orignal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install scikit-plot
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from sklearn.model_selection import *
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot
import gc
from tensorflow.keras import regularizers
from sklearn.preprocessing import minmax_scale
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve as learning_cv
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import roc_curve, auc
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import validation_curve
import scikitplot as skplt   
from sklearn.metrics import classification_report   
import re
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_curve 
from sklearn.metrics import auc
import seaborn as sns
from sklearn import metrics
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score,confusion_matrix
import re
%matplotlib inline
import os
# import call method from subprocess module 
from subprocess import call 

from google.colab import drive
drive.mount('/content/drive')

import bz2
import re
#from gensim.parsing.preprocessing import remove_stopwords

#import spacy
from numba import jit, cuda 
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
y_train, train_texts= get_labels_and_texts('/content/drive/My Drive/test.ft.txt.bz2')
y_train, train_texts = y_train[:100000], train_texts[:100000]
print("data readed")
gc.collect()
xx=[]
import re
for i in train_texts:
    i=i.lower()
    i=i.replace('.',' ')
    i=i.replace('`',' ')
    i=i.replace('~',' ')
    i=i.replace('@',' ')
    i=i.replace('#',' ')
    i=i.replace('$',' ')
    i=i.replace('%',' ')
    i=i.replace('^',' ')
    i=i.replace('&',' ')
    i=i.replace('*',' ')
    i=i.replace('(',' ')
    i=i.replace(')',' ')
    i=i.replace('_',' ')
    i=i.replace('-',' ')
    i=i.replace('+',' ')
    i=i.replace('=',' ')
    i=i.replace('{',' ')
    i=i.replace('}',' ')
    i=i.replace('[',' ')
    i=i.replace(']',' ')
    i=i.replace('|',' ')
    i=i.replace("\\",' ')
    i=i.replace(':',' ')
    i=i.replace(';',' ')
    i=i.replace('\"',' ')
    i=i.replace('/',' ')

    i=i.replace('?',' ')
    i=i.replace('<',' ')
    i=i.replace(',',' ')
    i=re.sub("n't",' not',i)
    i=re.sub("'ll",' will',i)
    i=re.sub("'m",' am',i)
    i=re.sub("'s"," is",i)
    i=re.sub("'re"," are",i)
    i=re.sub("'ve"," have",i)

    i=re.sub("'d"," would",i)
    i=re.sub("'"," ",i)
    i=i.replace(r"\n"," ")

    i=i.replace('>',' ')
    i=i.replace('!',' ')


    i=i.replace('>',' ')
    i=i.replace('_',' ')
    i=i.replace(' br ',' ')
    i=re.sub(' +',' ',i)
    i=re.sub('\b[a-z]+\b',' ',i)
    xx.append(i)
x_traint=xx
del xx
gc.collect()
print("data refine done")

# imported the requests library 
import requests 
data_url = "https://github.com/MukulKirtiVerma/Negtive-Positive-Word-List/raw/master/Positive%20and%20Negative%20Word%20List.xlsx"

# URL of the image to be downloaded is defined as image_url 
r = requests.get(data_url) # create HTTP response object 

# send a HTTP request to the server and save 
# the HTTP response in a response object called r 
with open("data_file.xlsx",'wb') as f: 

	# Saving received content as a png file in 
	# binary format 

	# write the contents of the response (r.content) 
	# to a new file in binary mode. 
	f.write(r.content) 
df=pd.read_excel('/content/data_file.xlsx')
df.head()
pw=list(df['Positive Sense Word List'])
nw=list(df['Negative Sense Word List'])

print('pw nw read')

def load_data(Train_df,y,idx,batch_size):
    global fl
    global model_w
    global nw
    global pw
    global word_data
    x_t=[]
    for i in range(idx*batch_size,idx*batch_size+batch_size):
        try:
            #print(i,len(Train_df),len(y),idx,batch_size)
            w=np.zeros((100,len(fl)))
            train=Train_df[i]
            train=train.split(' ')
            for j in train:

                if j in fl:
                    if j in nw:
                        w[:,fl.index(j)]=np.asarray(minmax_scale(model_w.wv.word_vec(j),feature_range=(-1,-0.9999)))    
                    elif j in pw:
                        w[:,fl.index(j)]=np.asarray(minmax_scale(model_w.wv.word_vec(j),feature_range=(0,1)))    
                    else:
                        w[:,fl.index(j)]=np.asarray(model_w.wv.word_vec(j))
        except: 
            break
        x_t.append(w)
        
    return (np.asarray(x_t), np.asarray(y[idx*batch_size:idx*batch_size+batch_size]))
def batch_generator(Train_df,y,batch_size,steps):
    idx=1
    while True: 
        yield load_data(Train_df,y,idx-1,batch_size)## Yields data
        print(idx,steps,end='')
        if idx<steps:
            idx+=1
        else:
            idx=1
            break

word_dict={}
for i in x_traint:
    for word in i.split(' '):
        try:
            word_dict[word]=word_dict[word]+1
        except:
            word_dict[word]=1
sort_orders = sorted(word_dict.items(), key=lambda x: x[1], reverse=True)
s_d=dict(sort_orders)


fl=[]
for i in s_d:
    if(s_d[i]<=8):
        if(i in pw or i in nw):
            fl.append(i)
    else:
        fl.append(i)
print(len(fl))


word_data=[]
word_data=[i.split(' ') for i in x_traint]
print("word 2 vec started")

import numpy as np
# train model
model_w = Word2Vec(word_data,size=100, min_count=1)
#x_train=word_to_vec_with_polarity(x_train,model_w)
gc.collect()
print('word2vec done')

def load_data_glove(Train_df,y,idx,batch_size):
    global fl
    global model_g
    global nw
    global pw
    global word_data
    x_t=[]
    for i in range(idx*batch_size,idx*batch_size+batch_size):
        try:
            #print(i,len(Train_df),len(y),idx,batch_size)
            w=np.zeros((100,len(fl)))
            train=Train_df[i]
            train=train.split(' ')
            for j in train: 
                try:
                    if j in fl:
                        if j in nw:
                            w[:,fl.index(j)]=np.asarray(minmax_scale(model_g[j],feature_range=(-1,-0.9999)))    
                        elif j in pw:
                            w[:,fl.index(j)]=np.asarray(minmax_scale(model_g[j],feature_range=(0,1)))    
                        else:
                            w[:,fl.index(j)]=np.asarray(model_g[j])
                except:
                    pass
        except: 
            break
        x_t.append(w)
        
    return (np.asarray(x_t), np.asarray(y[idx*batch_size:idx*batch_size+batch_size]))
def batch_generator_glove(Train_df,y,batch_size,steps):
    idx=1
    while True: 
        yield load_data_glove(Train_df,y,idx-1,batch_size)## Yields data
        print(idx,steps,end='')
        if idx<steps:
            idx+=1
        else:
            idx=1
            break
print('glove started')

from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = '/content/drive/My Drive/imdb analysis/glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
model_g = KeyedVectors.load_word2vec_format(filename, binary=False)
print('glove model fit')

from tensorflow.keras import regularizers


batch_size=70
training_size=int((len(y_train)*70)/100)
val_size=int((training_size*20)/100)
test_size=int((len(y_train)*30)/100)
steps_per_epoch=int(training_size/batch_size)
val_steps_per_epoch=int(val_size/batch_size)
test_steps_per_epoch=int(test_size/batch_size)
x_step=val_steps_per_epoch

def training_process(model):
    global y_train
    global x_traint
    global x_step
    batch_size=70
    training_size=int((len(y_train)*70)/100)
    val_size=int((training_size*20)/100)
    test_size=int((len(y_train)*30)/100)
    steps_per_epoch=int(training_size/batch_size)
    val_steps_per_epoch=int(val_size/batch_size)
    test_steps_per_epoch=int(test_size/batch_size)
    
    x_train=batch_generator(x_traint[:training_size],y_train[:training_size],batch_size,steps_per_epoch)
    x_val=batch_generator(x_traint[training_size:training_size+val_size],y_train[training_size:training_size+val_size],batch_size,x_step)
    history=model.fit(x_train,
     epochs=1,steps_per_epoch=steps_per_epoch, 
     verbose=1, validation_data=x_val,
     validation_steps=val_steps_per_epoch)
    # Plot the training loss 
    #plt.plot(history.history['accuracy'])
    # Plot the validation loss
    #plt.plot(history.history['val_accuracy'])
    # Show the figure
    #plt.show()
    # Plot the training loss 
    #plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    # Show the figure
    #plt.show()
    return model

In [ ]:
def training_process_glove(model):
    global y_train
    global x_traint
    global x_step

    batch_size=70
    training_size=int((len(y_train)*70)/100)
    val_size=int((training_size*20)/100)
    test_size=int((len(y_train)*30)/100)
    steps_per_epoch=int(training_size/batch_size)
    val_steps_per_epoch=int(val_size/batch_size)
    test_steps_per_epoch=int(test_size/batch_size)

    """
    training_size=28000
    val_size=7000
    test_size=15000"""
    
    
    x_train=batch_generator_glove(x_traint[:training_size],y_train[:training_size],batch_size,steps_per_epoch)
    x_val=batch_generator_glove(x_traint[training_size:training_size+val_size],y_train[training_size:training_size+val_size],batch_size,x_step)
    history=model.fit_generator(x_train,
     epochs=1,steps_per_epoch=steps_per_epoch,
     verbose=1, validation_data=x_val,
     validation_steps=val_steps_per_epoch)
    # Plot the training loss 
    plt.plot(history.history['accuracy'])
    # Plot the validation loss
    plt.plot(history.history['val_accuracy'])
    # Show the figure
    plt.show()
    # Plot the training loss 
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    # Show the figure
    plt.show()
    return model

def ploting(y_pred,y_test,y_proba,title):
    #confmatrix
    #confmatrix
    #ax = plt.axes()
    if(type(y_pred)==type([1,2,3])):
      y_pred=np.asarray(y_pred)
    confusion_mat=confusion_matrix(y_test,y_pred)
    print("TPR :" ,confusion_mat[0][0]/(confusion_mat[0][0]+confusion_mat[0][1]))
    print("FPR :" ,confusion_mat[1][0]/(confusion_mat[1][0]+confusion_mat[1][1]))
    m = tf.keras.metrics.Precision()
    _ = m.update_state(y_test,y_pred.reshape(y_test.shape[0],-1))
    precision=m.result().numpy()
    m = tf.keras.metrics.Recall()
    _ = m.update_state(y_test,y_pred.reshape(y_test.shape[0],-1))
    recall=m.result().numpy() 
    print("precision :", precision)
    print("recall :", recall)
    print('Accuracy score:{}'.format(accuracy_score(y_test, y_pred)))
    print('F1 score :{}'.format(f1_score(y_test, y_pred)))
    print('ROC AUC score:{}'.format(roc_auc_score(y_test, y_pred)))
    print(classification_report(y_test, y_pred))
    ax = plt.axes()
    confusion_mat=confusion_matrix(y_test,y_pred)
    sns.heatmap(confusion_mat, annot=True, fmt="d",ax = ax)
    ax.set_title('Confusion matrix with '+title+'\n')
    plt.show() 
    skplt.metrics.plot_roc(y_test, y_proba,plot_micro=False) 
    plt.title("ROC Curve with "+title+"\n")
    plt.figure(figsize=(30,30))
    plt.show()
    skplt.metrics.plot_precision_recall(y_test, y_proba)
    plt.title("Precision Recall Curve with " +title+"\n")
    plt.show()
    
test_len=0
test_len2=0
def test_result(model):
    global y_train
    global x_traint
    batch_size=70
    training_size=int((len(y_train)*70)/100)
    val_size=int((training_size*20)/100)
    test_size=int((len(y_train)*30)/100)
    steps_per_epoch=int(training_size/batch_size)
    val_steps_per_epoch=int(val_size/batch_size)
    test_steps_per_epoch=int(test_size/batch_size)

    x_train=batch_generator(x_traint[:training_size],y_train[:training_size],batch_size,steps_per_epoch)
    x_test=batch_generator(x_traint[training_size:],
                           y_train[training_size:],
                           batch_size,
                           test_steps_per_epoch)
    x_val=batch_generator(x_traint[training_size:training_size+val_size],
                          y_train[training_size:training_size+val_size],
                          batch_size,
                          800)
    preds = model.predict_generator(x_test)
    print(len(preds),end='')
    global test_len
    global test_len2
    test_len=len(preds)
    test_len2=test_size
    test_labels=y_train[training_size:len(preds)-test_size]
    y_pred=1 * (preds > 0.5)
    """
    print('Accuracy score:{}'.format(accuracy_score(test_labels, y_pred)))
    print('F1 score:{}'.format(f1_score(test_labels, y_pred)))
    print('ROC AUC score:{}'.format(roc_auc_score(test_labels, y_pred)))"""
    print(classification_report(test_labels, y_pred))
    y_proba=[]
    for i in preds:
        y_proba.append([1-i[0],i[0]])
    return test_labels,y_pred,y_proba



In [ ]:
def test_result_glove(model):
    global y_train
    global x_traint
    batch_size=70
    training_size=int((len(y_train)*70)/100)
    val_size=int((training_size*20)/100)
    test_size=int((len(y_train)*30)/100)
    steps_per_epoch=int(training_size/batch_size)
    val_steps_per_epoch=int(val_size/batch_size)
    test_steps_per_epoch=int(test_size/batch_size)
    x_train=batch_generator_glove(x_traint[:training_size],y_train[:training_size],batch_size,steps_per_epoch)
    x_test=batch_generator_glove(x_traint[training_size:],
                           y_train[training_size:],
                           batch_size,
                           test_steps_per_epoch)
    x_val=batch_generator_glove(x_traint[training_size:training_size+val_size],
                          y_train[training_size:training_size+val_size],
                          batch_size,
                          800)
    preds = model.predict_generator(x_test)
    test_labels=y_train[training_size:len(preds)-test_size]    
    y_pred=1 * (preds > 0.5)
    """
    print('Accuracy score:{}'.format(accuracy_score(test_labels, y_pred)))
    print('F1 score:{}'.format(f1_score(test_labels, y_pred)))
    print('ROC AUC score:{}'.format(roc_auc_score(test_labels, y_pred)))"""
    print(classification_report(test_labels, y_pred))
    y_proba=[]
    for i in preds:
        y_proba.append([1-i[0],i[0]])
    return test_labels,y_pred,y_proba,

In [ ]:
epochs=1
model = layers.Input(shape=(100, len(fl)))
x = layers.Conv1D(64, 3,activation='relu')(model)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(3)(x)
x = layers.Conv1D(64, 5, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(64, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Flatten()(x)
x = layers.Dense(100, activation='relu',
                  bias_regularizer=regularizers.l1(1e-3), 
                  activity_regularizer=regularizers.l2(1e-2))(x)
predictions = layers.Dense(1, activation='sigmoid',
                            bias_regularizer=regularizers.l1(1e-3),
                            activity_regularizer=regularizers.l1(1e-3))(x)
model1 = models.Model(inputs=model, outputs=predictions)
model1.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model1=training_process(model1)
  training_acc.extend(model1.history.history['accuracy'])
  val_acc.extend(model1.history.history['val_accuracy'])
  training_loss.extend(model1.history.history['loss'])
  val_loss.extend(model1.history.history['val_loss'])
# summarize history for accuracy

plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch') 
plt.legend(['train', 'test'], loc='upper left')
plt.show()
y_test1,y_pred1,y_proba1=test_result(model1)
#print(y_test,y_pred,y_proba) 
ploting(y_pred1,y_test1,y_proba1,'Conv + word2vec Embedding')
print('Model1 exicuted')


model6 = layers.Input(shape=(100, len(fl)))
x = layers.Conv1D(64, 3,activation='relu')(model6)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(3)(x)
x = layers.Conv1D(64, 5, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(64, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Flatten()(x)
x = layers.Dense(100, activation='relu',
                  bias_regularizer=regularizers.l1(1e-3), 
                  activity_regularizer=regularizers.l2(1e-2))(x)

predictions = layers.Dense(1, activation='sigmoid', 

                  bias_regularizer=regularizers.l1(1e-3),
                  activity_regularizer=regularizers.l1(1e-3))(x)
model6 = models.Model(inputs=model6, outputs=predictions)
model6.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model6=training_process_glove(model6)
  training_acc.extend(model6.history.history['accuracy'])
  val_acc.extend(model6.history.history['val_accuracy'])
  training_loss.extend(model6.history.history['loss'])
  val_loss.extend(model6.history.history['val_loss'])
# summarize history for accuracy
"""
plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()"""

y_test6,y_pred6,y_proba6=test_result_glove(model6)
#print(y_test,y_pred,y_proba)
ploting(y_pred6,y_test6,y_proba6,'Conv + Glove Embedding')
"""
y_test6,y_pred6,y_proba6=test_result_glove(model6)
#print(y_test,y_pred,y_proba) 
ploting(y_pred6,y_test6,y_proba6,'Conv + Glove Embedding')"""
print('Model 2 (6) exicuted')



model2 = layers.Input(shape=(100,len(fl)))
x = layers.GRU(128, return_sequences=True)(model2)
x = layers.GRU(128)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)
model2 = models.Model(inputs=model2, outputs=predictions)
model2.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model2=training_process(model2)
  training_acc.extend(model2.history.history['accuracy'])
  val_acc.extend(model2.history.history['val_accuracy'])
  training_loss.extend(model2.history.history['loss'])
  val_loss.extend(model2.history.history['val_loss'])
# summarize history for accuracy
"""
plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()"""

y_test2,y_pred2,y_proba2=test_result(model2) 
#print(y_test,y_pred,y_proba) 
ploting(y_pred2,y_test2,y_proba2,'GRU + word2vec Embedding')
print('model 3 (2)exicuted')


model3 = layers.Input(shape=(100,len(fl)))
x = layers.LSTM(128, return_sequences=True)(model3)
x = layers.LSTM(128)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)
model3 = models.Model(inputs=model3, outputs=predictions)
model3.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model3=training_process(model3)
  training_acc.extend(model3.history.history['accuracy'])
  val_acc.extend(model3.history.history['val_accuracy'])
  training_loss.extend(model3.history.history['loss'])
  val_loss.extend(model3.history.history['val_loss'])
# summarize history for accuracy
"""
plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()"""
y_test3,y_pred3,y_proba3=test_result(model3)

#print(y_test,y_pred,y_proba) 
ploting(y_pred3,y_test3,y_proba3,'LSTM + word2vec Embedding')
print('model 4 (3) exicuted :')

model7 = layers.Input(shape=(100,len(fl)))
x = layers.GRU(128, return_sequences=True)(model7)
x = layers.GRU(128)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)
model7 = models.Model(inputs=model7, outputs=predictions)
model7.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model7=training_process_glove(model7)
  training_acc.extend(model7.history.history['accuracy'])
  val_acc.extend(model7.history.history['val_accuracy'])
  training_loss.extend(model7.history.history['loss'])
  val_loss.extend(model7.history.history['val_loss'])
# summarize history for accuracy
"""
plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()"""
y_test7,y_pred7,y_proba7=test_result_glove(model7)
#print(y_test,y_pred,y_proba)
ploting(y_pred7,y_test7,y_proba7,'GRU + Glove Embedding')
print('Model 5(7) exicuted')

model8 = layers.Input(shape=(100,len(fl)))
x = layers.LSTM(128, return_sequences=True)(model8)
x = layers.LSTM(128)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)
model8 = models.Model(inputs=model8, outputs=predictions)
model8.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model8=training_process_glove(model8)
  training_acc.extend(model8.history.history['accuracy'])
  val_acc.extend(model8.history.history['val_accuracy'])
  training_loss.extend(model8.history.history['loss'])
  val_loss.extend(model8.history.history['val_loss'])
# summarize history for accuracy
"""
plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()"""
y_test8,y_pred8,y_proba8=test_result_glove(model8)
#print(y_test,y_pred,y_proba)
ploting(y_pred8,y_test8,y_proba8 ,'LSTM + Glove Embedding')
print('model 6 (8) exicuted')

model4 = layers.Input(shape=(100,len(fl)))
x = layers.Bidirectional(layers.GRU(128, return_sequences=True))(model4)
x = layers.Bidirectional(layers.GRU(128))(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x) 
model4 = models.Model(inputs=model4, outputs=predictions)
model4.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model4=training_process(model4)
  training_acc.extend(model4.history.history['accuracy'])
  val_acc.extend(model4.history.history['val_accuracy'])
  training_loss.extend(model4.history.history['loss'])
  val_loss.extend(model4.history.history['val_loss'])
# summarize history for accuracy
"""
plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()"""
y_test4,y_pred4,y_proba4=test_result(model4)
#print(y_test,y_pred,y_proba) 
ploting(y_pred4,y_test4,y_proba4,'Bidirectional GRU + word2vec Embedding')
print('model 7 (4) exicuted')

model9 = layers.Input(shape=(100,len(fl))) 
x = layers.Bidirectional(layers.GRU(128, return_sequences=True))(model9)
x = layers.Bidirectional(layers.GRU(128))(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)
model9 = models.Model(inputs=model9, outputs=predictions)
model9.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model9=training_process_glove(model9)
  training_acc.extend(model9.history.history['accuracy'])
  val_acc.extend(model9.history.history['val_accuracy'])
  training_loss.extend(model9.history.history['loss'])
  val_loss.extend(model9.history.history['val_loss'])
# summarize history for accuracy
"""
plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()"""
y_test9,y_pred9,y_proba9=test_result_glove(model9)
#print(y_test,y_pred,y_proba)
ploting(y_pred9,y_test9,y_proba9,'Bidirectional GRU + Glove Embedding')
print('model 8 (9) exicuted')


model5 = layers.Input(shape=(100,len(fl)))
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(model5)
x = layers.Bidirectional(layers.LSTM(128))(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)
model5 = models.Model(inputs=model5, outputs=predictions)
model5.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model5=training_process(model5)
  training_acc.extend(model5.history.history['accuracy'])
  val_acc.extend(model5.history.history['val_accuracy'])
  training_loss.extend(model5.history.history['loss'])
  val_loss.extend(model5.history.history['val_loss'])
# summarize history for accuracy
"""
plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()"""
y_test5,y_pred5,y_proba5=test_result(model5)
#print(y_test,y_pred,y_proba)  
ploting(y_pred5,y_test5,y_proba5,'Bidirectional LSTM + word2vec Embedding')

print('model 9(5) exicuted')


model10 = layers.Input(shape=(100,len(fl)))
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(model10)
x = layers.Bidirectional(layers.LSTM(128))(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)
model10 = models.Model(inputs=model10, outputs=predictions)
model10.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
training_acc=[]
training_loss=[]
val_acc=[]
val_loss=[]
for i in range(epochs):
  model10=training_process_glove(model10)
  training_acc.extend(model10.history.history['accuracy'])
  val_acc.extend(model10.history.history['val_accuracy'])
  training_loss.extend(model10.history.history['loss'])
  val_loss.extend(model10.history.history['val_loss'])
# summarize history for accuracy
"""
plt.plot(training_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()"""
y_test10,y_pred10,y_proba10=test_result_glove(model10)
#print(y_test,y_pred,y_proba)
ploting(y_pred10,y_test10,y_proba10,'Bidirectional LSTM + Glove Embedding')
print('model 10 (10) exicuted')


from sklearn.feature_extraction.text import CountVectorizer
st=CountVectorizer(stop_words='english',max_features=28000,lowercase=True) 
st.fit(x_traint) 
x_train_c= st.fit_transform(x_traint)

from sklearn import linear_model
estimator = linear_model.SGDClassifier(loss='log')
estimator.fit(x_train_c[:70000], y_train[:70000])
from sklearn.metrics import accuracy_score
y_pred11 = estimator.predict(x_train_c[70000:])
y_proba11=estimator.predict_proba(x_train_c[70000:])
from sklearn.metrics import classification_report   #for calculating precision recall f1-score abd support
print(classification_report(y_train[70000:test_len-test_len2],y_pred11[:test_len-test_len2]))
ax = plt.axes()

y_test11=y_train[70000:test_len-test_len2] 
ploting(y_pred11[:test_len-test_len2],y_test11,y_proba11[:test_len-test_len2],'SGD + Count Vectorizer')
final_y_pred=[]
final_y_proba=[]

for i in range(len(y_pred2)):
    final_y_proba.append([(y_proba1[i][0]+y_proba2[i][0]+y_proba3[i][0]+y_proba4[i][0]+y_proba5[i][0]+y_proba6[i][0]+y_proba7[i][0]+y_proba8[i][0]+y_proba9[i][0]+y_proba10[i][0]+y_proba11[i][0])/11
                          ,(y_proba1[i][1]+y_proba2[i][1]+y_proba3[i][1]+y_proba4[i][1]+y_proba5[i][1]+y_proba6[i][1]+y_proba7[i][1]+y_proba8[i][1]+y_proba9[i][1]+y_proba10[i][1]+y_proba11[i][1])/11])
    x_n=sum(y_pred1[i]+y_pred2[i]+y_pred3[i]+y_pred4[i]+y_pred5[i]+y_pred6[i]+y_pred7[i]+y_pred8[i]+y_pred9[i]+y_pred10[i]+y_pred11[i])

    if (x_n>=6):
        final_y_pred.append(1)
    else:
        final_y_pred.append(0)
ploting(final_y_pred,y_train[70000:test_len-test_len2],final_y_proba,'Proposed Ensemble Classifier')
probas_list=[y_proba1,y_proba2,y_proba3,y_proba4,y_proba5 ,y_proba6,y_proba7,y_proba8,y_proba9,y_proba10,y_proba11,final_y_proba]
clf_names=['Conv+Word2vec+polarity','Conv+Glove+polarity','GRU+Word2vec+polarity','GRU+Glove+polarity',
           'LSTM+Word2vec+polarity','LSTM+Glove+polarity','Bidirectional GRU+Word2vec+polarity','Bidirectional GRU+Glove+polarity',
           'Bidirectional LSTM+Word2vec+polarity','Bidirectional LSTM+Glove+polarity','SGD+Count Vectorizer','Final ensemble Model']

skplt.metrics.plot_calibration_curve(y_train[70000:test_len-test_len2],probas_list,clf_names,figsize=(10,10))
j=0
for i in probas_list:
  skplt.metrics.plot_cumulative_gain(y_train[70000:test_len-test_len2], i,title='Cumulative Gain Curve For : ' +clf_names[j])
  j+=1
plt.show() 

j=0
for i in probas_list:
  skplt.metrics.plot_lift_curve(y_train[70000:test_len-test_len2], i,title='Lift Curve for : '+clf_names[j])
  j+=1
plt.show() 